In [1]:
%load_ext autoreload
%autoreload 2

import cv2
import yaml
from tqdm import tqdm
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import Image, display
from typing import List
import gradio as gr

/opt/conda/envs/diffinv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [44]:
import torch
torch.set_grad_enabled(False)

from demo.run import EditorManager


editor_manager = EditorManager()


models = [
    ("CompVis/stable-diffusion-v1-4", "SD-1.4"),
]

inverters = [
    ("diffinv", "Diffusion inversion"),
    ("nti", "Null-text inversion"),
    ("npi", "Negative prompt inversion"),
    ("proxnpi", "Proximal negative prompt inversion"),
    ("edict", "EDICT"),
    ("ddpminv", "DDPM inversion"),
]


editors = [
    ("simple", "Simple"),
    ("ptp", "Prompt-to-prompt"),
    ("masactrl", "MasaControl"),
    ("pnp", "Plug-and-play"),
    ("pix2pix_zero", "Pix2pix zero"),
]


def name_to_value(lst, name):
    out = next((v for v, n in lst if n == name), None)
    assert out is not None, f"{name} not found in {lst}"
    return out


def value_to_name(lst, value):
    out = next((n for v, n in lst if v == value), None)
    assert out is not None, f"{value} not found {lst}"
    return out


def get_model_choices():
    return list(zip(*models))[1]


def get_inverter_choices(model):
    if model is None:
        return []

    model = name_to_value(models, model)
  
    if model in ("CompVis/stable-diffusion-v1-4", ):
        out = ["diffinv", "nti", "npi", "proxnpi", "edict", "ddpminv"]
    else:
        out = []

    return [value_to_name(inverters, val) for val in out]


def get_editor_choices(inverter):
    if inverter is None:
        return []
    
    inverter = name_to_value(inverters, inverter) if inverter is not None else None

    out = ["simple", "ptp", "masactrl", "pnp", "pix2pix_zero"]

    return [value_to_name(editors, val) for val in out]
    

def run(cfg):
    cfg["model.type"] = name_to_value(models, cfg["model.type"])
    cfg["inverter.type"] = name_to_value(inverters, cfg["inverter.type"])
    cfg["editor.type"] = name_to_value(editors, cfg["editor.type"])

    edit_res = editor_manager.run(cfg)
    
    return edit_res["edit_image"]


def update_inversion_config_visibility(names, cat):
    cat = name_to_value(inverters, cat)
    return [gr.update(visible=name.startswith(f"inverter.methods.{cat}.")) for name in names]


default_values = {
    "model": value_to_name(models, "CompVis/stable-diffusion-v1-4"),
    "inverter": value_to_name(inverters, "diffinv"),
    "editor": value_to_name(editors, "simple"),
}

# TODO: add gallery with old images

# TODO: auto generate prompt and fill textbox
with gr.Blocks() as demo:
    inputs = {}
    outputs = {}

    with gr.Row():
        inputs["edit_cfg.source_image"] = gr.Image(label="Input", value="test/data/gnochi_mirror_sq.png", width=512, height=512)
        outputs["edit_image"] = gr.Image(label="Output", width=512, height=512)

    with gr.Row():
        with gr.Column():
            inputs["editor.type"] = gr.Dropdown(label="Edit method", choices=get_editor_choices(default_values["inverter"]), value=default_values["editor"], interactive=True)
            inputs["edit_cfg.source_prompt"] = gr.Textbox(label="Source prompt", value="a cat sitting next to a mirror")
            inputs["edit_cfg.target_prompt"] = gr.Textbox(label="Target prompt", value="a tiger sitting next to a mirror")

        with gr.Column():
            with gr.Row():
                inputs["model.type"] = gr.Dropdown(label="Model", choices=get_model_choices(), value=default_values["model"])

            with gr.Row():
                inputs["inverter.type"] = gr.Dropdown(label="Inversion method", choices=get_inverter_choices(default_values["model"]), value=default_values["inverter"], interactive=True)

                for inverter, inverter_name in inverters:
                    visible = default_values["inverter"] == inverter_name
                    inputs[f"inverter.methods.{inverter}.num_inference_steps"] = gr.Number(label="Steps", value=50, precision=0, visible=visible)
                    inputs[f"inverter.methods.{inverter}.guidance_scale_fwd"] = gr.Number(label="Forward CFG scale", value=1.0, visible=visible)
                    inputs[f"inverter.methods.{inverter}.guidance_scale_bwd"] = gr.Number(label="Backward CFG scale", value=7.5, visible=visible)
                    inputs[f"inverter.methods.{inverter}.scheduler"] = gr.Dropdown(label="Scheduler", choices=["ddim", "dpm"], value="ddim", visible=visible)
    
    inputs["model.type"].change(lambda model: gr.update(choices=get_inverter_choices(model)), inputs=inputs["model.type"], outputs=inputs["inverter.type"])
    inputs["inverter.type"].change(lambda inverter: gr.update(choices=get_editor_choices(inverter)), inputs=inputs["inverter.type"], outputs=inputs["editor.type"])

    inversion_inputs = {k: v for k, v in inputs.items() if k.startswith("inverter.methods.")}
    inputs["inverter.type"].change(lambda value: update_inversion_config_visibility(list(inversion_inputs.keys()), value), inputs=inputs["inverter.type"], outputs=list(inversion_inputs.values()))

    edit_btn = gr.Button("Edit")
    edit_btn.click(fn=lambda *values: run(dict(zip(inputs.keys(), values))), inputs=list(inputs.values()), outputs=outputs["edit_image"])

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7901
Running on public URL: https://08867b678aab7db5a9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  4.43it/s]
